In [2]:
import pandas as pd
import requests

In [3]:
# Nasdaq Stock Screener is a good way to get a (full) list of stocks, 7626 stocks as of June 22 2023
# https://www.nasdaq.com/market-activity/stocks/screener
# You can download the csv file manually
data = pd.read_csv("nasdaq_screener_20230622.csv")
data.sort_values(by="Market Cap", ascending=False, inplace=True)
data.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
22,AAPL,Apple Inc. Common Stock,$185.465,1.5050,0.818%,2.917123e+12,United States,1980.0,7281925,Technology,Computer Manufacturing
4540,MSFT,Microsoft Corporation Common Stock,$336.71,3.1500,0.944%,2.503603e+12,United States,1986.0,3687698,Technology,Computer Software: Prepackaged Software
3050,GOOG,Alphabet Inc. Class C Capital Stock,$121.605,0.3450,0.285%,1.544019e+12,United States,2004.0,3221308,Technology,Computer Software: Programming Data Processing
3051,GOOGL,Alphabet Inc. Class A Common Stock,$120.94,0.3900,0.324%,1.535575e+12,United States,2004.0,5474863,Technology,Computer Software: Programming Data Processing
419,AMZN,Amazon.com Inc. Common Stock,$127.9599,3.1299,2.507%,1.312914e+12,United States,1997.0,15850686,Consumer Discretionary,Catalog/Specialty Distribution


In [4]:
# Or make a HTTP request and convert the JSON payload to a dataframe
# Request URL can ben found in Network tab in Chrome
# User agent is used to identify the application, operating system, vendor, and/or version of the requesting user agent to custom code for broswer variants
# Using the correct user agent when performing data scraping tasks is crucial while avoding being blocked
# https://brightdata.com/blog/how-tos/user-agents-for-web-scraping-101
payload = requests.get('https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true', headers={'User-agent': 'Chrome/114.0.0.0'}).json()
data = pd.json_normalize(payload["data"]["rows"])
data.rename(columns=payload["data"]["headers"], inplace=True)
data["Market Cap"] = pd.to_numeric(data["Market Cap"]) # to_numeric handle empty string conversion
data.sort_values(by="Market Cap", ascending=False, inplace=True)
data.head()

,Symbol,Name,Last Sale,Net Change,% Change,Volume,Market Cap,Country,IPO Year,Industry,Sector,Url
22,AAPL,Apple Inc. Common Stock,$187.00,3.04,1.653%,51157733,2.941267e+12,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl
4613,MSFT,Microsoft Corporation Common Stock,$339.71,6.15,1.844%,23530167,2.525909e+12,United States,1986,Computer Software: Prepackaged Software,Technology,/market-activity/stocks/msft
3102,GOOG,Alphabet Inc. Class C Capital Stock,$123.87,2.61,2.152%,20764851,1.572777e+12,United States,2004,Computer Software: Programming Data Processing,Technology,/market-activity/stocks/goog
3103,GOOGL,Alphabet Inc. Class A Common Stock,$123.15,2.60,2.157%,26917461,1.563636e+12,United States,2004,Computer Software: Programming Data Processing,Technology,/market-activity/stocks/googl
421,AMZN,Amazon.com Inc. Common Stock,$130.15,5.32,4.262%,90154037,1.335385e+12,United States,1997,Catalog/Specialty Distribution,Consumer Discretionary,/market-activity/stocks/amzn


In [5]:
# % Change/Market Cap/Country/IPO Year/Sector/Industry are not populated for some stocks (real world data!)
# Apart from 11 GICS sectors, some stocks have Miscellaneous and '' populated as Sector
data["Sector"].unique()

array(['Technology', 'Consumer Discretionary', 'Utilities', '',
       'Health Care', 'Energy', 'Finance', 'Consumer Staples',
       'Telecommunications', 'Industrials', 'Basic Materials',
       'Real Estate', 'Miscellaneous'], dtype=object)

In [6]:
us_data = data[(data["Country"] == "United States")]
us_data.head(10)

,Symbol,Name,Last Sale,Net Change,% Change,Volume,Market Cap,Country,IPO Year,Industry,Sector,Url
22,AAPL,Apple Inc. Common Stock,$187.00,3.04,1.653%,51157733,2.941267e+12,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl
4613,MSFT,Microsoft Corporation Common Stock,$339.71,6.15,1.844%,23530167,2.525909e+12,United States,1986,Computer Software: Prepackaged Software,Technology,/market-activity/stocks/msft
3102,GOOG,Alphabet Inc. Class C Capital Stock,$123.87,2.61,2.152%,20764851,1.572777e+12,United States,2004,Computer Software: Programming Data Processing,Technology,/market-activity/stocks/goog
3103,GOOGL,Alphabet Inc. Class A Common Stock,$123.15,2.60,2.157%,26917461,1.563636e+12,United States,2004,Computer Software: Programming Data Processing,Technology,/market-activity/stocks/googl
421,AMZN,Amazon.com Inc. Common Stock,$130.15,5.32,4.262%,90154037,1.335385e+12,United States,1997,Catalog/Specialty Distribution,Consumer Discretionary,/market-activity/stocks/amzn
4993,NVDA,NVIDIA Corporation Common Stock,$430.25,-0.20,-0.046%,41690980,1.062718e+12,United States,1999,Semiconductors,Technology,/market-activity/stocks/nvda
6960,TSLA,Tesla Inc. Common Stock,$264.61,5.15,1.985%,166411396,8.386825e+11,United States,2010,Auto Manufacturing,Consumer Discretionary,/market-activity/stocks/tsla
5343,PCGU,Pacific Gas & Electric Co. Equity Unit,$149.80,0.51,0.342%,37729,8.178441e+11,United States,,Power Generation,Utilities,/market-activity/stocks/pcgu
1134,BRK/A,Berkshire Hathaway Inc.,$511480.00,-4035.00,-0.783%,6457,7.523226e+11,United States,,,,/market-activity/stocks/brk/a
1135,BRK/B,Berkshire Hathaway Inc.,$336.96,-1.65,-0.487%,3303233,7.434387e+11,United States,,,,/market-activity/stocks/brk/b
